In [95]:
import pandas as pd
import pytz
from datetime import datetime
from matplotlib import pyplot as plt

In [96]:
data = pd.read_csv('Sales-products-tz-mod.csv', index_col = 'SaleID')
sales_data = data.copy()
sales_data

,RetailerCountry,RetailerType,Product,Sales Revenue ($),DateOfSale,TimeOfSale,TimeZone
SaleID,,,,,,,
SaleID_1,United States,Outdoors Shop,TrailChef Deluxe Cook Set,200.0,06/01/2020,23:20:56,EST
SaleID_2,United States,Outdoors Shop,TrailChef Double Flame,7.0,05/02/2020,17:27:08,EST
SaleID_3,United States,Outdoors Shop,Star Dome,20.0,30/10/2020,09:04:43,EST
SaleID_4,United States,Outdoors Shop,Star Gazer 2,40.0,13/11/2020,04:25:06,EST
SaleID_5,Italy,Outdoors Shop,Canyon Mule Carryall,150.5,06/12/2020,11:15:47,CET
...,...,...,...,...,...,...,...
SaleID_96,Australia,Sports Store,Mountain Man Extreme,24.0,23/07/2019,12:30:03,Australia/West
SaleID_97,Australia,Department Store,Firefly Mapreader,1200.0,03/01/2019,01:39:14,Australia/West
SaleID_98,Australia,Discount Retailer,Polar Sun,32.0,14/02/2019,21:29:35,Australia/West


In [97]:
sales_data[["DateOfSale","TimeOfSale","TimeZone"]].head()

,DateOfSale,TimeOfSale,TimeZone
SaleID,,,
SaleID_1,06/01/2020,23:20:56,EST
SaleID_2,05/02/2020,17:27:08,EST
SaleID_3,30/10/2020,09:04:43,EST
SaleID_4,13/11/2020,04:25:06,EST
SaleID_5,06/12/2020,11:15:47,CET


## Part 1 - data preparation

- **Subtask 1:** Store the date and time values in a single column called 'MOS', denoting 'Moment of Sale'. Verify that the time zone values stored in "sales_data" are valid and can be manipulated with the pytz module. *(Data Preparation)*

In [98]:
date_and_time = sales_data["DateOfSale"] + " " + sales_data["TimeOfSale"]
date_and_time.head(10)

SaleID
SaleID_1     06/01/2020 23:20:56
SaleID_2     05/02/2020 17:27:08
SaleID_3     30/10/2020 09:04:43
SaleID_4     13/11/2020 04:25:06
SaleID_5     06/12/2020 11:15:47
SaleID_6     07/06/2020 01:54:41
SaleID_7     27/12/2020 02:07:28
SaleID_8     15/09/2020 11:44:57
SaleID_9     23/09/2020 01:40:29
SaleID_10    24/06/2020 18:43:04
dtype: object

In [99]:
sales_data["MOS"] = pd.to_datetime(date_and_time,format="%d/%m/%Y %H:%M:%S")
sales_data.head(10)

,RetailerCountry,RetailerType,Product,Sales Revenue ($),DateOfSale,TimeOfSale,TimeZone,MOS
SaleID,,,,,,,,
SaleID_1,United States,Outdoors Shop,TrailChef Deluxe Cook Set,200.00,06/01/2020,23:20:56,EST,2020-01-06 23:20:56
SaleID_2,United States,Outdoors Shop,TrailChef Double Flame,7.00,05/02/2020,17:27:08,EST,2020-02-05 17:27:08
SaleID_3,United States,Outdoors Shop,Star Dome,20.00,30/10/2020,09:04:43,EST,2020-10-30 09:04:43
SaleID_4,United States,Outdoors Shop,Star Gazer 2,40.00,13/11/2020,04:25:06,EST,2020-11-13 04:25:06
SaleID_5,Italy,Outdoors Shop,Canyon Mule Carryall,150.50,06/12/2020,11:15:47,CET,2020-12-06 11:15:47
SaleID_6,Italy,Outdoors Shop,Firefly 4,1300.00,07/06/2020,01:54:41,CET,2020-06-07 01:54:41
SaleID_7,United Kingdom,Outdoors Shop,Husky Rope 50,270.00,27/12/2020,02:07:28,GMT,2020-12-27 02:07:28
SaleID_8,United Kingdom,Outdoors Shop,Granite Signal Mirror,499.99,15/09/2020,11:44:57,GMT,2020-09-15 11:44:57
SaleID_9,United Kingdom,Outdoors Shop,Granite Carabiner,32.00,23/09/2020,01:40:29,GMT,2020-09-23 01:40:29


In [100]:
# to verify that the timezones are valid and can be manipulated by pytz
sales_data["TimeZone"].unique() # SGT is the one that could cause troubles

array(['EST', 'CET', 'GMT', 'EET', 'SGT', 'Australia/West'], dtype=object)

In [101]:
# pytz.timezone("SGT") #==> causes an error
troublesom_timezones = set() #using a set to store just a single value
for i in sales_data["TimeZone"]:
  if i not in pytz.all_timezones:
    troublesom_timezones.add(i)
print(troublesom_timezones)

{'SGT'}


SGT is not in the pytz module, hence, it should be modified. First, DateOfSale and TimeOfSale are dropped to clean the dataframe

In [102]:
sales_data.drop(columns=["TimeOfSale","DateOfSale"],axis=1, inplace=True)
sales_data.head()

,RetailerCountry,RetailerType,Product,Sales Revenue ($),TimeZone,MOS
SaleID,,,,,,
SaleID_1,United States,Outdoors Shop,TrailChef Deluxe Cook Set,200.0,EST,2020-01-06 23:20:56
SaleID_2,United States,Outdoors Shop,TrailChef Double Flame,7.0,EST,2020-02-05 17:27:08
SaleID_3,United States,Outdoors Shop,Star Dome,20.0,EST,2020-10-30 09:04:43
SaleID_4,United States,Outdoors Shop,Star Gazer 2,40.0,EST,2020-11-13 04:25:06
SaleID_5,Italy,Outdoors Shop,Canyon Mule Carryall,150.5,CET,2020-12-06 11:15:47


In [103]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, SaleID_1 to SaleID_100
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   RetailerCountry    100 non-null    object        
 1   RetailerType       100 non-null    object        
 2   Product            100 non-null    object        
 3   Sales Revenue ($)  100 non-null    float64       
 4   TimeZone           100 non-null    object        
 5   MOS                100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 5.5+ KB


In [104]:
sales_data[(sales_data["TimeZone"]=="SGT")]

,RetailerCountry,RetailerType,Product,Sales Revenue ($),TimeZone,MOS
SaleID,,,,,,
SaleID_18,Singapore,Outdoors Shop,Granite Pulley,19.00,SGT,2019-01-02 01:32:09
SaleID_19,Singapore,Outdoors Shop,Firefly Climbing Lamp,23.45,SGT,2019-02-20 07:07:10
SaleID_20,Singapore,Outdoors Shop,Granite Ice,65.00,SGT,2019-04-10 10:27:26
SaleID_21,Singapore,Outdoors Shop,Mountain Man Analog,10.00,SGT,2019-07-10 05:20:29
SaleID_22,Singapore,Outdoors Shop,Mountain Man Digital,12.00,SGT,2020-05-03 09:54:41
SaleID_23,Singapore,Outdoors Shop,Mountain Man Deluxe,19.00,SGT,2019-07-25 06:51:57
SaleID_81,Singapore,Warehouse Store,EverGlow Kerosene,23.06,SGT,2019-11-10 12:50:36
SaleID_82,Singapore,Outdoors Shop,TX,50.00,SGT,2019-09-25 19:48:39


In [105]:
replaced_timezones = sales_data.loc[:,"TimeZone"].replace("SGT", "Singapore")
replaced_timezones[replaced_timezones=="Singapore"]

SaleID
SaleID_18    Singapore
SaleID_19    Singapore
SaleID_20    Singapore
SaleID_21    Singapore
SaleID_22    Singapore
SaleID_23    Singapore
SaleID_81    Singapore
SaleID_82    Singapore
Name: TimeZone, dtype: object

In [106]:
sales_data["TimeZone"] = replaced_timezones
sales_data[(sales_data["TimeZone"]=="Singapore")]

,RetailerCountry,RetailerType,Product,Sales Revenue ($),TimeZone,MOS
SaleID,,,,,,
SaleID_18,Singapore,Outdoors Shop,Granite Pulley,19.00,Singapore,2019-01-02 01:32:09
SaleID_19,Singapore,Outdoors Shop,Firefly Climbing Lamp,23.45,Singapore,2019-02-20 07:07:10
SaleID_20,Singapore,Outdoors Shop,Granite Ice,65.00,Singapore,2019-04-10 10:27:26
SaleID_21,Singapore,Outdoors Shop,Mountain Man Analog,10.00,Singapore,2019-07-10 05:20:29
SaleID_22,Singapore,Outdoors Shop,Mountain Man Digital,12.00,Singapore,2020-05-03 09:54:41
SaleID_23,Singapore,Outdoors Shop,Mountain Man Deluxe,19.00,Singapore,2019-07-25 06:51:57
SaleID_81,Singapore,Warehouse Store,EverGlow Kerosene,23.06,Singapore,2019-11-10 12:50:36
SaleID_82,Singapore,Outdoors Shop,TX,50.00,Singapore,2019-09-25 19:48:39


# Part 2 - Data Manipulation

- **Subtask 2:** Estimate the offset of the values of "MOS" to UTC. Store them in a column called "OffsetUTC". *(Data Manipulation)*

In [107]:
sales_data["MOS"].iloc[0]

Timestamp('2020-01-06 23:20:56')

In [108]:
sales_data["MOS"].iloc[0].tz_localize(sales_data["TimeZone"].iloc[0])


Timestamp('2020-01-06 23:20:56-0500', tz='EST')

In [109]:
sales_data.shape

(100, 6)

As it is not recommended to modify directly, I create a list of the new dates with the offset based on timezone. That list will be added to the dataframe `sales_data`

In [110]:
list_of_offset_dates = []

for i in range(len(sales_data)):
  list_of_offset_dates.append(pd.to_datetime(sales_data["MOS"].iloc[i].tz_localize(sales_data["TimeZone"].iloc[i])))
list_of_offset_dates

[Timestamp('2020-01-06 23:20:56-0500', tz='EST'),
 Timestamp('2020-02-05 17:27:08-0500', tz='EST'),
 Timestamp('2020-10-30 09:04:43-0500', tz='EST'),
 Timestamp('2020-11-13 04:25:06-0500', tz='EST'),
 Timestamp('2020-12-06 11:15:47+0100', tz='CET'),
 Timestamp('2020-06-07 01:54:41+0200', tz='CET'),
 Timestamp('2020-12-27 02:07:28+0000', tz='GMT'),
 Timestamp('2020-09-15 11:44:57+0000', tz='GMT'),
 Timestamp('2020-09-23 01:40:29+0000', tz='GMT'),
 Timestamp('2020-06-24 18:43:04+0200', tz='CET'),
 Timestamp('2019-05-19 02:12:30+0200', tz='CET'),
 Timestamp('2019-04-05 07:38:10-0500', tz='EST'),
 Timestamp('2020-01-24 08:51:28-0500', tz='EST'),
 Timestamp('2019-12-07 08:14:53-0500', tz='EST'),
 Timestamp('2020-01-13 05:32:40-0500', tz='EST'),
 Timestamp('2020-11-26 10:17:48+0200', tz='EET'),
 Timestamp('2020-01-23 19:36:25+0200', tz='EET'),
 Timestamp('2019-01-02 01:32:09+0800', tz='Singapore'),
 Timestamp('2019-02-20 07:07:10+0800', tz='Singapore'),
 Timestamp('2019-04-10 10:27:26+0800',

The offset is an integer number representing the difference from the UTC time

In [111]:
list_of_offset_dates[0]

Timestamp('2020-01-06 23:20:56-0500', tz='EST')

In [112]:
list_of_offset_dates[0].strftime("%z")

'-0500'

In [113]:
list_of_offset_dates[0].utcoffset() # ==> returns the total offset form the UTC

datetime.timedelta(days=-1, seconds=68400)

In [114]:
list_of_offset_dates[0].utcoffset().total_seconds()/3600 # ==> the real number of hours offset

-5.0

In [115]:
float(list_of_offset_dates[0].strftime("%z"))/100 # ==> the offset is -5.0 hours, YOU CAN USE BOTH

-5.0

In [119]:
offset_hours = []
for i in range(len(list_of_offset_dates)):
  offset_hours.append(float(list_of_offset_dates[i].strftime("%z"))/100)
offset_hours

[-5.0,
 -5.0,
 -5.0,
 -5.0,
 1.0,
 2.0,
 0.0,
 0.0,
 0.0,
 2.0,
 2.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 2.0,
 2.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 2.0,
 3.0,
 2.0,
 3.0,
 3.0,
 3.0,
 2.0,
 1.0,
 0.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 -5.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0,
 1.0,
 2.0,
 1.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 8.0,
 8.0,
 1.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0,
 8.0]

In [120]:
sales_data["OffsetUTC"] = offset_hours
sales_data

,RetailerCountry,RetailerType,Product,Sales Revenue ($),TimeZone,MOS,OffsetUTC
SaleID,,,,,,,
SaleID_1,United States,Outdoors Shop,TrailChef Deluxe Cook Set,200.0,EST,2020-01-06 23:20:56,-5.0
SaleID_2,United States,Outdoors Shop,TrailChef Double Flame,7.0,EST,2020-02-05 17:27:08,-5.0
SaleID_3,United States,Outdoors Shop,Star Dome,20.0,EST,2020-10-30 09:04:43,-5.0
SaleID_4,United States,Outdoors Shop,Star Gazer 2,40.0,EST,2020-11-13 04:25:06,-5.0
SaleID_5,Italy,Outdoors Shop,Canyon Mule Carryall,150.5,CET,2020-12-06 11:15:47,1.0
...,...,...,...,...,...,...,...
SaleID_96,Australia,Sports Store,Mountain Man Extreme,24.0,Australia/West,2019-07-23 12:30:03,8.0
SaleID_97,Australia,Department Store,Firefly Mapreader,1200.0,Australia/West,2019-01-03 01:39:14,8.0
SaleID_98,Australia,Discount Retailer,Polar Sun,32.0,Australia/West,2019-02-14 21:29:35,8.0


In [121]:
# Just reordering columns

column_names_reordered = ['RetailerCountry', 'MOS', 'TimeZone', 'OffsetUTC', 'RetailerType', 'Product', 'Sales Revenue ($)']
sales_data = sales_data[column_names_reordered]
sales_data.head()

,RetailerCountry,MOS,TimeZone,OffsetUTC,RetailerType,Product,Sales Revenue ($)
SaleID,,,,,,,
SaleID_1,United States,2020-01-06 23:20:56,EST,-5.0,Outdoors Shop,TrailChef Deluxe Cook Set,200.0
SaleID_2,United States,2020-02-05 17:27:08,EST,-5.0,Outdoors Shop,TrailChef Double Flame,7.0
SaleID_3,United States,2020-10-30 09:04:43,EST,-5.0,Outdoors Shop,Star Dome,20.0
SaleID_4,United States,2020-11-13 04:25:06,EST,-5.0,Outdoors Shop,Star Gazer 2,40.0
SaleID_5,Italy,2020-12-06 11:15:47,CET,1.0,Outdoors Shop,Canyon Mule Carryall,150.5


In [128]:
sales_data.groupby(["TimeZone","OffsetUTC"]).mean("Sales Revenue ($)").sort_values("Sales Revenue ($)",ascending=False)

,,Sales Revenue ($)
TimeZone,OffsetUTC,
EET,2.0,944.875000
CET,2.0,517.821818
Australia/West,8.0,273.105882
EST,-5.0,230.385500
EET,3.0,228.500000
CET,1.0,191.064000
GMT,0.0,115.137273
Singapore,8.0,27.688750
